In [4]:
import matplotlib.pyplot as plt
import numpy as np

inputs = np.array([])

with open("input.txt", "r") as file:
    for line in file:
        splitted = line.split( )
        inputs = np.append(inputs,splitted)

n = int(inputs[0])

colors = np.random.rand(n,3)
print(colors)

muArrival , sigmaArrival = float(inputs[1]), float(inputs[2])
muBurst , sigmaBurst = float(inputs[3]), float(inputs[4])
lamda = float(inputs[5])

Arrivals = np.random.normal(muArrival,sigmaArrival ,n)
Burst = np.random.normal(muBurst,sigmaBurst ,n)
Priority = np.random.poisson(lamda,n)
Waiting = np.zeros(Arrivals.shape)

contextSwitching = 5

fh = open("output.txt","w")

for i in range(n):
    fh.write(str(i+1) + ' ' + str(Arrivals[i]) + ' ' + str(Burst[i]) + ' ' + str(Priority[i]) + '\n')

fh.close()


processes = np.array([range(n),Arrivals,Burst,Priority,Waiting]).transpose()
print('Original:')
print(processes)
print(' ')


[[0.34513298 0.58797778 0.79833736]
 [0.42267563 0.13394152 0.04303564]
 [0.77244385 0.60616569 0.11088247]
 [0.97028279 0.25309041 0.21664909]
 [0.40241898 0.30111747 0.23608327]
 [0.91333067 0.77706863 0.71841027]
 [0.25199872 0.82063914 0.53549538]]
Original:
[[ 0.          8.69702181  8.46922552  7.          0.        ]
 [ 1.          8.85880933  6.82859473  6.          0.        ]
 [ 2.          7.55397179  6.21451667  6.          0.        ]
 [ 3.         10.26956416  6.46921238 11.          0.        ]
 [ 4.          8.85794421 20.98497757  5.          0.        ]
 [ 5.          5.57623416 10.50180333  8.          0.        ]
 [ 6.          7.9801881  21.09300054  8.          0.        ]]
 


# Non-Preemptive Highest Priority First (HPF)

In [5]:

temp = processes[processes[:,3].argsort()] [::-1]
#processes = processes[np.lexsort(processes[:,::-1,:].T)]
print('After Arranging:')
print(temp)
order = []

currentTime = 0

while temp.shape[0] != 0:
    found = False
    for idx, i in enumerate(temp):
        if i[1] < currentTime:
            processes[int(i[0]),4] = currentTime - i[1]
            order += [i[0]]
            currentTime += i[2]
            currentTime += contextSwitching
            temp = np.delete(temp,idx,0)
            found = True
            break
    if (found == False):
        currentTime += 1


print(order)

print(currentTime)


print(processes)

After Arranging:
[[ 3.         10.26956416  6.46921238 11.          0.        ]
 [ 6.          7.9801881  21.09300054  8.          0.        ]
 [ 5.          5.57623416 10.50180333  8.          0.        ]
 [ 0.          8.69702181  8.46922552  7.          0.        ]
 [ 2.          7.55397179  6.21451667  6.          0.        ]
 [ 1.          8.85880933  6.82859473  6.          0.        ]
 [ 4.          8.85794421 20.98497757  5.          0.        ]]
[5.0, 3.0, 6.0, 0.0, 2.0, 1.0, 4.0]
121.56133074558286
[[ 0.          8.69702181  8.46922552  7.         50.36699444]
 [ 1.          8.85880933  6.82859473  6.         74.88894912]
 [ 2.          7.55397179  6.21451667  6.         64.97926998]
 [ 3.         10.26956416  6.46921238 11.         11.23223917]
 [ 4.          8.85794421 20.98497757  5.         86.71840896]
 [ 5.          5.57623416 10.50180333  8.          0.42376584]
 [ 6.          7.9801881  21.09300054  8.         24.99082761]]


# Create Graph

In [7]:
import matplotlib, sys
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
if sys.version_info[0] < 3:
    import Tkinter as Tk
else:
    import tkinter as Tk

root = Tk.Tk()

f = Figure(figsize=(20,8), dpi=70)

currentTime = processes[int(order[0]),1]

for i in order:
    
    ax = f.add_subplot(111)
    data = (i+1)

    ind = (currentTime) 
    width = processes[int(i),2]
    ind += 0.5*width

    rects = ax.bar(ind, data, width, alpha = 0.8,color=colors[int(data)-1] )
    currentTime += processes[int(i),2]
    currentTime += contextSwitching
     


ax.set_xlabel('Time')
ax.set_ylabel('Process')
ax.set_title('HPF Sceduling Algorithm')
patchList = []

for idx,color in enumerate(colors):
    data_key = matplotlib.patches.Patch(color=color, label='P'+str(idx+1))
    patchList.append(data_key)
    
ax.legend(handles=patchList)

ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
ax.xaxis.set_ticks(np.arange(0, currentTime, 4))
ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
ax.yaxis.set_ticks(np.arange(0, n+1, 1))



canvas = FigureCanvasTkAgg(f, master=root)
canvas.show()
canvas.get_tk_widget().pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)

Tk.mainloop()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\ProgramData\A

# Round-Robin with Quantam 

In [8]:
import matplotlib, sys
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
if sys.version_info[0] < 3:
    import Tkinter as Tk
else:
    import tkinter as Tk

temp = processes[processes[:,1].argsort()]

queue = []
remainingTime = []
quantam = 1.0
currentTime = temp[0][1]

print(currentTime)

root = Tk.Tk()

f = Figure(figsize=(20,8), dpi=70)
j=0

while len(queue) != 0 or temp.shape[0] != 0:
    print(queue)
    if (len(queue) != 0):
        
        
        ax = f.add_subplot(111)
        data = queue[j%len(queue)] +1
        ind = (currentTime) 
        
        if (remainingTime[j%len(remainingTime)] < quantam ):
            currentTime += (remainingTime[j%len(remainingTime)])
            width = remainingTime[j%len(remainingTime)]
            del queue[j%len(queue)]
            del remainingTime[j%len(remainingTime)]
        else:
            remainingTime[j%len(remainingTime)] -= quantam
            currentTime += quantam
            width = quantam
            j += 1
                            
        currentTime += contextSwitching
        
        ind += 0.5*width

        rects = ax.bar(ind, data, width, alpha = 0.8,color=colors[int(data-1)]  )

    #check if new process came

    if temp.shape[0] != 0:
        found = False
        for idx, i in enumerate(temp):
            if i[1] < currentTime:
                processes[int(i[0]),4] = currentTime - i[1]
                queue += [i[0]]
                remainingTime += [i[2]]
                temp = np.delete(temp,idx,0)
                found = True
                break
        if (found == False and len(queue) == 0):
            currentTime += 0.01
        
        
        
        
        
        
        
        
        
        
ax.set_xlabel('Time')
ax.set_ylabel('Process')
ax.set_title('Round Robin')




patchList = []

for idx,color in enumerate(colors):
    data_key = matplotlib.patches.Patch(color=color, label='P'+str(idx+1))
    patchList.append(data_key)
    
ax.legend(handles=patchList)



ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
ax.xaxis.set_ticks(np.arange(0, currentTime, quantam))

ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%g'))
ax.yaxis.set_ticks(np.arange(0, n+1, 1))


canvas = FigureCanvasTkAgg(f, master=root)
canvas.show()
canvas.get_tk_widget().pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)

Tk.mainloop()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\ProgramData\A

5.576234164838298
[]
[]
[5.0]
[5.0, 2.0]
[5.0, 2.0, 6.0]
[5.0, 2.0, 6.0, 0.0]
[5.0, 2.0, 6.0, 0.0, 4.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[5.0, 2.0, 6.0, 0.0, 4.0, 1.0, 3.0]
[

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: MatplotlibDeprecationWarning: The FigureCanvasTk.show function was deprecated in version 2.2. Use FigureCanvasTk.draw instead.
